In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import matplotlib.pyplot as plt
import torch.nn.init as init
import time

In [2]:
def np_to_torch(arr):
    
    arr = torch.FloatTensor(arr)
    arr = arr.unsqueeze(-1)
    arr = arr.clone().detach().requires_grad_(True)
    
    return arr

def x_train_data(N, x_l, x_r):
    
    x_train = np.linspace(x_l,x_r,N)   
    x_train = np.concatenate((x_train,np.ones(1)*x_r,np.ones(1)*x_l),0)
    x_train = np_to_torch(x_train)
    return x_train

def initial_temp(N_tot, T_l, T_r):
    
    T_prev = np.ones(N_tot)*T_r
    T_prev[0] = T_l
    T_prev = np_to_torch(T_prev)
    
    return T_prev

def initial_interface(N, s):

    s_interface = torch.full((N, 1), s)
    s_interface = s_interface.clone().detach().requires_grad_(True)
    
    return s_interface, s_interface
            
def xavier_init(m):
    if isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight)
        if m.bias is not None:
            init.constant_(m.bias, 0)
    
class ANN(nn.Module):
    def __init__(self, layer_size):
        super(ANN, self).__init__()
        
        # Fully conected model
        modules = []
        for i in range(len(layer_size) - 2):
            modules.append(nn.Linear(layer_size[i], layer_size[i+1]))  
            modules.append(nn.Tanh())
        modules.append(nn.Linear(layer_size[-2], layer_size[-1])) 

        self.fc = nn.Sequential(*modules)
#         for layer in self.fc.modules():
#             if isinstance(layer, nn.Linear):
#                  layer.weight.data.normal_(mean=0, std=0.2)
        self.fc.apply(xavier_init)
        
    def forward(self, x_train, s):
        
        T = self.fc( x_train )
        dTdx = torch.autograd.grad(T, x_train, grad_outputs=torch.ones_like(T), create_graph=True)[0]
        d2Tdx2 = torch.autograd.grad(dTdx, x_train, grad_outputs=torch.ones_like(dTdx), create_graph=True)[0]
        Ts = self.fc( s )
        
        return T, dTdx, d2Tdx2, Ts
    
def get_loss(x_train, k1, k2, T_l, T_r, x_l, x_r, T_prev, del_t, t_test, s, N_eq1):
    
    mse = nn.MSELoss(reduction='sum')
    w1 = 1
    w2 = 1
    w3 = 1
        
    T, dTdx, d2Tdx2, Ts  = model(x_train, s)
    eq1 = w1*( torch.sum( torch.square( torch.mul(torch.where(x_train <= s,1,0),T - T_prev - del_t*k1*d2Tdx2 ) ) ) )/(N_eq1)
    bc1 = w2*torch.sum( torch.square( torch.mul(torch.where(x_train == x_l,1,0),(T - T_l)) ) )
    bc2 = w3*torch.sum( torch.square( T_r - Ts ) )

    loss = eq1 + bc1 + bc2   
    
    return loss, eq1, bc1, bc2

def print_loss(epoch, loss, eq1, bc1, bc2):
    print('epoch = ',epoch)
    print('loss = ',loss.detach().numpy())
    print('eq1_loss = ',eq1.detach().numpy())
    print('bc1_loss = ',bc1.detach().numpy())
    print('bc2_loss = ',bc2.detach().numpy())

def interface_condition(f, del_t, k2, T, interface_index, T_r, del_x, N_colpts, N_tot, dTdx, d2Tdx2):
    
#     print("Double Derivative = ", d2Tdx2[interface_index - N_colpts - 1][0])
#     f_new = f + del_t*k2*( d2Tdx2[interface_index - N_colpts - 1][0] )
#     f_new = f + del_t*k2*( T[interface_index - N_colpts - 1][0] - T_r )/del_x**2
#     f_new = f + del_t*k2*( -dTdx[interface_index - N_colpts - 1][0])/del_x
    f_new = f + del_t*k2*( -dTdx[interface_index + int(f*N_colpts - N_colpts/2)][0])/del_x
    
    if(f_new>1):
        T_new = (f_new - 1)*k1/k2 + T_r
        T[interface_index][0] = T_new
        print("Interface changed, temperature at the interface =  ", T[interface_index][0])     
        interface_index += N_colpts + 1
        f_new = 0
#         f_new = f_new-1
#         f_new = 0
#         f_new = del_t*k2*( T_new - T_r )/del_x**2
    
    for j in range(interface_index + int(f_new*N_colpts - N_colpts/2), N_tot):
        T[j][0] = T_r
        
    return torch.FloatTensor(T), interface_index, f_new
    
def lamb_analytical(k1, k2):
    x = []
    er = []
    cnt = 0
    for i in np.arange(0.1, 5, 0.001):
        x.append(i)
        er.append(math.erf(x[-1]))
        cnt = cnt+1

    x = np.array(x)
    er = np.array(er)
    y =[]
    y = np.exp(-x*x)/(er*math.sqrt(math.pi))-x*k1/k2

    for i in range(1,cnt):
        if(y[i]*y[i-1]<0):
            lam = x[i]
            break
    
    return lam

def analytical(N_x_test, x_test, t_test, T_r, k1, k2, T_l):

    x_test = x_test.detach().numpy()
    y_an = np.zeros((N_x_test, 1))
    lam = lamb_analytical(k1, k2)
    s = np.sqrt(k1*t_test)*2*lam
    
    for j in range(N_x_test):
        if(x_test[j]<s):
            y_an[j] = T_l - T_l*math.erf( x_test[j]/( 2*np.sqrt(k1*t_test) ) )/ math.erf(lam) 
        else:
            y_an[j] = T_r
            
    y_an = np.reshape(y_an, (N_x_test, 1))
    
    return y_an, s
    
def train_model(model, optimiser1, epochs, T_r, T_l, k1, k2, N_x, x_l, x_r, N_t, N_bc, accuracy_cap, N_x_test, del_t, s_initial, del_x):
    
    loss_store = []
    T_store_pred = []
    s_store_pred = []
    T_store_an = []
    s_store_an = []
    t_store = []
    mse = nn.MSELoss(reduction='sum')
    model.train()  
    
    N_gridpts = int((x_r - x_l)/del_x + 1)
    print("gridpoints = ",N_gridpts)
    N_colpts = 13
    N_tot = (N_colpts + 1)*(N_gridpts - 1) + 1 
    print("N_tot = ", N_tot)
    
    t_test = 0
    t_store.append(0)
    s_store_an.append(0)
    
    #Initial conditions
    interface_index = N_colpts + 1
    x_train = x_train_data(N_tot, x_l, x_r)
    T_prev = initial_temp(N_tot, T_l, T_r)
    s_prev = np_to_torch( x_train[interface_index].detach().numpy() )
    f_liq = 0
    for i in range(N_t):
        
        t_test = t_test + del_t
        t_store.append(t_test)
        print("t = ", t_test)
        print(" ")
        N_eq1 = interface_index + 1
        
        if i>2:
            epochs = 1501
            
        print("epochs = ", epochs)
        for epoch in range(epochs):
            
            #Backpropogation and optimisation
            loss, eq1, bc1, bc2 = get_loss(x_train, k1, k2, T_l, T_r, x_l, x_r, T_prev, del_t, t_test, x_train[interface_index + int(f_liq*N_colpts - N_colpts/2)], N_eq1)
            optimiser1.zero_grad()
            loss.backward()
            optimiser1.step()  
            loss_store.append(loss.detach().numpy())
            
            if epoch%2000==0:
                print_loss(epoch, loss, eq1, bc1, bc2)
                print("")

#             if loss<accuracy_cap :
#                 print("loss limit attained, epoch = ", epoch)
#                 break
                    
        # Store the results after each time step
        T_prev, dTdx_prev, d2Tdx2_prev,_ = model(x_train, s_prev)
        T_prev, interface_index, f_liq = interface_condition(f_liq, del_t, k2, T_prev.detach().numpy(), interface_index, T_r, del_x, N_colpts, N_tot, dTdx_prev.detach().numpy(), d2Tdx2_prev.detach().numpy())
        s_prev = torch.ones(1,1)*x_train[interface_index + int(f_liq*N_colpts - N_colpts/2)][0].detach().numpy().item()
        print("f_liq = ", f_liq)
        print("interface_PINN = ", s_prev[0][0].detach().numpy())
        T_an, s_an = analytical(N_tot, x_train, t_test, T_r, k1, k2, T_l)
        print("interface_Analytical = ", s_an)
        
        T_store_pred.append(T_prev.detach().numpy())
        T_store_an.append(T_an)
        s_store_pred.append(s_prev.detach().numpy())
        s_store_an.append(s_an)
        
        T_prev = torch.FloatTensor(T_store_pred[-1]).clone().detach().requires_grad_(False)
        s_prev = torch.FloatTensor(s_store_pred[-1]).clone().detach().requires_grad_(True)
        
        print("broke inner loop")
        print("")

    return loss_store, T_store_pred, T_store_an, x_train, s_store_pred, s_store_an, t_store

In [3]:
N_x = 3001
N_bc = 30
N_t = 150
del_t = 0.001
del_x = 0.01
x_l = 0
x_r = 0.2
T_r = 0
T_l = 1
t_i = 0
accuracy_cap = 0.0004
N_x_test = 251
s_initial = 0.0015

# Neural network params
layer_size = [1, 3, 3, 3, 1]

# material params
k1 = 0.01
k2 = 0.005

# Training data and initial data
model = ANN(layer_size)
print(model)
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total trainable parameters in the model:", total_trainable_params)

# # Setup Loss function and Optimiser
lr = 2e-4
epochs = 50001
optimiser1 = torch.optim.Adam(model.parameters(), lr=lr)

# Training model
start = time.time()
loss_store, T_store_pred, T_store_an, x_test_np, s_pred, s_an, t = train_model(model, optimiser1, epochs, T_r, T_l, k1, k2, N_x, x_l, x_r, N_t, N_bc, accuracy_cap, N_x_test, del_t, s_initial, del_x)
end = time.time()
time_elapsed = end - start
print("time elapsed = ", time_elapsed)

ANN(
  (fc): Sequential(
    (0): Linear(in_features=1, out_features=3, bias=True)
    (1): Tanh()
    (2): Linear(in_features=3, out_features=3, bias=True)
    (3): Tanh()
    (4): Linear(in_features=3, out_features=3, bias=True)
    (5): Tanh()
    (6): Linear(in_features=3, out_features=1, bias=True)
  )
)
Total trainable parameters in the model: 34
gridpoints =  21
N_tot =  281
t =  0.001
 
epochs =  50001


RuntimeError: The size of tensor a (283) must match the size of tensor b (281) at non-singleton dimension 0

# Results Plotter

In [ ]:
j = 149
i = 0
k = 301

plt.plot(x_test_np[i:k].detach().numpy(), T_store_pred[j][i:k])
plt.plot(x_test_np[i:k].detach().numpy(), T_store_an[j][i:k])
Title = "Stefan 1-phase using Time stepping, " + "time = " + str("{:.3f}".format((j+1)*del_t)) + ", delta_t = " + str("{:.3f}".format(del_t))
plt.title(Title)
plt.legend(["PINN", "Analytical"])
plt.show()

In [ ]:
tp = []
for i in range(len(s_pred)):
    tp.append(s_pred[i][0])

i = 0
j = 149
plt.plot(t[i:j], tp[i:j])
plt.plot(t[i:j], s_an[i:j])
plt.legend(["PINN", "Analytical"])
plt.xlabel('Time')
plt.ylabel('Interface position')
plt.show()

In [ ]:
plt.plot(loss_store)